In [ ]:
# This cell installs the Unsloth library.  
# This notebook contains codes provided by Unsloth.ai.

%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Enable 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # The model we are using
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
# Apply Parameter-Efficient Fine-Tuning (PEFT) using LoRA

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    # LoRA-applied modules
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407, # for reproducibility
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ (another quantization method)
)

Unsloth 2024.12.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# This is for formatting dataset examples into the required prompt format

import pandas as pd

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# examples = pd.read_csv("/content/train_llama_format.csv")

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# Loading our reformatted training set
dataset = load_dataset("csv", data_files="claim_only/finetuning/train_llama_format.csv", split = "train")

# Apply text formatting to our dataset
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/86367 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Initialize Trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    # data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.

    # Defining training arguments
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 500,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(), # Use FP16 if BF16 is not supported
        bf16 = is_bfloat16_supported(), # Use BF16 if supported
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        # report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/86367 [00:00<?, ? examples/s]

In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.635 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train() # Train the model

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 86,367 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 500
 "-____-"     Number of trainable parameters = 24,313,856
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.873500
2,2.897900
3,2.897500
4,2.740900
5,2.565500
6,2.217800
7,1.980700
8,1.737200
9,1.541000
10,1.281800


In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1538.5655 seconds used for training.
25.64 minutes used for training.
Peak reserved memory = 3.229 GB.
Peak reserved memory for training = 0.594 GB.
Peak reserved memory % of max memory = 21.894 %.
Peak reserved memory for training % of max memory = 4.028 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Tokenize input for model inference
# This is an example with a specific claim
inputs = tokenizer(
[
    prompt.format(
        '''You are an advanced AI fact-checker trained on a vast corpus, including information from Wikipedia. You are given a claim, and your task is to verify whether all the facts in the claim are supported by your knowledge base. Use your understanding of the world and factual data to make an accurate judgment. Your response must follow the given format exactly as specified below, without any deviation.
### Response Format: Your response should be a single word answer "True" or "False

## TASK:
Verify the given input claim.

### Instructions:
Assess whether the claim is true or false based on your knowledge.

### Answer Format:
Provide your response as a single word answer "True" or "False"''', # instruction
        "At least Dawn Butler had a successor!", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True) # generate outputs
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are an advanced AI fact-checker trained on a vast corpus, including information from Wikipedia. You are given a claim, and your task is to verify whether all the facts in the claim are supported by your knowledge base. Use your understanding of the world and factual data to make an accurate judgment. Your response must follow the given format exactly as specified below, without any deviation.\n### Response Format: Your response should be a single word answer "True" or "False\n\n## TASK:\nVerify the given input claim.\n\n### Instructions:\nAssess whether the claim is true or false based on your knowledge.\n\n### Answer Format:\nProvide your response as a single word answer "True" or "False"\n\n### Input:\nAt least Dawn Butler had a successor!\n\n### Response:\nTrue<|eot_id|>']

In [ ]:
import re
def get_model_prediction(claim):
  # this function is to get the prediction given the claim (if it is factual or not)
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference

  inputs = tokenizer(
  [
      prompt.format(
          '''You are an advanced AI fact-checker trained on a vast corpus, including information from Wikipedia. You are given a claim, and your task is to verify whether all the facts in the claim are supported by your knowledge base. Use your understanding of the world and factual data to make an accurate judgment. Your response must follow the given format exactly as specified below, without any deviation.
  ### Response Format: Your response should be a single word answer "True" or "False

  ## TASK:
  Verify the given input claim.

  ### Instructions:
  Assess whether the claim is true or false based on your knowledge.

  ### Answer Format:
  Provide your response as a single word answer "True" or "False"''', # instruction
          claim, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")


  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128) # Generate model prediction
  generated_answer = tokenizer.decode(_[0]) # Decode the generated output

  generated_answer = re.search(r'### Response:\s*(True|False)', generated_answer)

  if generated_answer:
    predicted_label = generated_answer.group(1)
  else:
    get_model_prediction(claim) # Call the function again until we get a valid response

  # print(type(predicted_label))
  stripped = predicted_label.strip().lower()
  if stripped == "true": # here i convert the true/false strings to boolean values
    predicted_label = True
    return predicted_label
  elif stripped == "false":
    predicted_label = False
  return predicted_label

In [ ]:
import csv
def write_to_csv(filename, claim, reasoning_type, true_label, predicted_label):
    # this function is to write the results to a csv file
    fieldnames = ["claim", "reasoning_type", "true_label", "predicted_label"]

    with open(filename, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # If the file is empty, write the header
        if file.tell() == 0:
            writer.writeheader()

        writer.writerow({"claim": claim, "reasoning_type": reasoning_type, "true_label": true_label, "predicted_label": predicted_label})

In [ ]:
def get_reasoning_type(claim, test_data):
    # This function is to get the reasoning type given the claim
    tags = test_data[claim]['types']
    resoning_types = []
    for tag in tags:
        if tag == 'negation':
            resoning_types.append('negation')
            break
        elif tag == 'num1':
            resoning_types.append('one-hop')
        elif tag == 'multi claim':
            resoning_types.append('conjuction')
        elif tag == 'existence':
            resoning_types.append('existence')
        elif tag == 'multi hop':
            resoning_types.append('multi hop')
    return resoning_types

In [ ]:
import pickle

with open('factkg_dataset/factkg_test.pickle', 'rb') as file:
    test_data = pickle.load(file)

# i = 1
# Iterate over the claims and their properties in the test dataset
for claim, information in test_data.items():
  # if i <= 1648:
  #   i += 1
    # continue
  reasoning_type = get_reasoning_type(claim, test_data)[0]
  true_label = information['Label'][0]
  predicted_label = get_model_prediction(claim)
  # Write the claim data to CSV for result analysis
  write_to_csv("claim_only/finetuning/finetuned_v0/fine_tuned_claim_only_v0.csv", claim, reasoning_type, true_label, predicted_label)

Streaming output truncated to the last 5000 lines.
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an advanced AI fact-checker trained on a vast corpus, including information from Wikipedia. You are given a claim, and your task is to verify whether all the facts in the claim are supported by your knowledge base. Use your understanding of the world and factual data to make an accurate judgment. Your response must follow the given format exactly as specified below, without any deviation.
  ### Response Format: Your response should be a single word answer "True" or "False

  ## TASK:
  Verify the given input claim.

  ### Instructions:
  Assess whether the claim is true or false based on your knowledge.

  ### Answer Format:
  Provide your response as a single word answer "True" or "False"

### Input:
Alfredo Zitarrosa death place was in 

In [ ]:
# This code cell is to get the achieved results

import pandas as pd
from sklearn.metrics import accuracy_score

df = pd.read_csv('claim_only/finetuning/finetuned_v0/fine_tuned_claim_only_v0.csv', dtype={'true_label': bool, 'predicted_label': bool})


accuracy = accuracy_score(df['true_label'], df['predicted_label'])


reasoning_type_accuracies = {'total': accuracy}
for reasoning_type, group in df.groupby('reasoning_type'):
    accuracy = accuracy_score(group['true_label'], group['predicted_label'])
    reasoning_type_accuracies[reasoning_type] = accuracy

print(reasoning_type_accuracies)

: 